<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# ! pip install fasttext

In [2]:
import os
import re
import csv
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [3]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [4]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
                break
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [5]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = read_dataset_from_file()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد
    </p>
</div>

In [7]:
def display_dataset_info():
    global CATEGORIES

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for data in dataset:
        length_dict[data['category-ENG']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,80986
1,سیاسی,15810
2,جهانی,4800
3,اقتصادی,9873
4,اجتماعی,14070
5,شهری,4260
6,ورزشی,8421
7,علمی,3267
8,فرهنگی,6657
9,فناوری اطلاعات,444


In [8]:
dataset = pd.DataFrame(dataset)
dataset

,date,title,intro,body,category-PER,category-ENG
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,"['سياست', 'سیاست\u200cخارجی']",Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,"['جهان', 'آمریکا']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,"['سياست', 'سیاست داخلی']",Politics
...,...,...,...,...,...,...
80981,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,"['زندگی', 'تندرستی']",LifeSkills
80982,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,"['زندگی', 'مهارت\u200cهای زندگی']",LifeSkills
80983,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,"['زندگی', 'پیشنهاد سفر']",LifeSkills
80984,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,"['زندگی', 'تندرستی']",LifeSkills


## Fasttext

In [9]:
train_text = ''
train_text += ' '.join(list(dataset.loc[:, 'title']))
train_text += ' '.join(list(dataset.loc[:, 'intro']))
train_text += ' '.join(list(dataset.loc[:, 'body']))
with open('test.txt', 'w', encoding='utf-8') as fp:
    fp.write(train_text)

In [10]:
dataset.loc[:100,]

,date,title,intro,body,category-PER,category-ENG
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,"['سياست', 'سیاست\u200cخارجی']",Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,"['جهان', 'آمریکا']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,"['سياست', 'سیاست داخلی']",Politics
...,...,...,...,...,...,...
96,یکشنبه ۲۹ فروردین ۱۴۰۰ - ۲۲:۱۹,عضو کارگزاران خطاب به فائزه هاشمی: تصور نکنید ...,یک عضو حزب کارگزاران سازندگی به‌تازگی در در وا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، چن...,"['سياست', 'سیاست داخلی']",Politics
97,یکشنبه ۲۹ فروردین ۱۴۰۰ - ۲۱:۵۴,جانشین فرمانده نیروی قدس درگذشت,روابط عمومی کل سپاه پاسداران انقلاب اسلامی در ...,به گزارش همشهری‌آنلاین به نقل از ایسنا، متن اط...,"['سياست', 'دفاع-امنیت']",Politics
98,یکشنبه ۲۹ فروردین ۱۴۰۰ - ۲۱:۳۰,تصمیم جدید بورسی| اجرای اصلاح دستورالعمل رفع گ...,هیات مدیره سازمان بورس اعلام کرد: اجرای مصوبه ...,به گزارش همشهری آنلاین به نقل از بازار سرمایه ...,"['اقتصاد', 'بورس']",Politics
99,یکشنبه ۲۹ فروردین ۱۴۰۰ - ۲۱:۱۰,داماد ترامپ؛ متهم کودتا در اردن!,رئیس مجلس اعیان اردن با اشاره به دخالت طرف‌های...,به گزارش همشهری آنلاین به نقل از فارس، رئیس مج...,"['جهان', 'غرب آسیا']",Politics


In [11]:
class FastText:
    
    def __init__(self, method="skipgram", k=5):
        self.model = None
        self.method = method
        self.k = k
    
    def train(self, dataset):
        # train_text = ''
        # train_text += ' '.join(list(dataset.loc[:100, 'title']))
        # train_text += ' '.join(list(dataset.loc[:100, 'intro']))
        # train_text += ' '.join(list(dataset.loc[:100, 'body']))
        
        self.model = fasttext.train_unsupervised('test.txt', self.method, minn=2, maxn=5)

    def predict(self, query, dataset):
        query_embed = np.mean([self.model.get_word_vector(x) for x in query], axis=0)
        dataset_sim = []
        for index, row in dataset.iterrows():
            title_embed = [self.model.get_word_vector(x) for x in row['title']]
            intro_embed = [self.model.get_word_vector(x) for x in row['intro']]
            body_embed = [self.model.get_word_vector(x) for x in row['body']]
            row_embed = np.mean(title_embed + intro_embed + body_embed, axis=0)
            row_cosine_sim = self.cosine_sim(query_embed, row_embed)
            dataset_sim.append(row_cosine_sim)
        idx = np.argsort(dataset_sim)
        return dataset.iloc[list(idx[:self.k])]

    def cosine_sim(self, query, row):
        return np.dot(query, row) / (np.linalg.norm(query) * np.linalg.norm(row))

    def save_ft_model(self, path="fasttext_model.bin"):
        self.model.save_model(path)
    
    def load_ft_model(self, path="fasttext_model.bin"):
        self.model = fasttext.load_model(path)

In [12]:
ft = FastText()
ft.train(dataset)
ft.save_ft_model()
ft.load_ft_model()

Read 37M words
Number of words:  114861
Number of labels: 0
Progress: 100.0% words/sec/thread:   44850 lr:  0.000000 avg.loss:  1.211225 ETA:   0h 0m 0s  0.6% words/sec/thread:   49873 lr:  0.049703 avg.loss:  1.771887 ETA:   0h 5m42s  2.5% words/sec/thread:   50840 lr:  0.048757 avg.loss:  1.687997 ETA:   0h 5m29s  5.2% words/sec/thread:   49799 lr:  0.047377 avg.loss:  1.718990 ETA:   0h 5m26s  5.7% words/sec/thread:   47999 lr:  0.047146 avg.loss:  1.729022 ETA:   0h 5m37s  9.0% words/sec/thread:   49969 lr:  0.045477 avg.loss:  1.751786 ETA:   0h 5m12s  9.2% words/sec/thread:   49999 lr:  0.045414 avg.loss:  1.751672 ETA:   0h 5m12s 10.0% words/sec/thread:   48708 lr:  0.045014 avg.loss:  1.757065 ETA:   0h 5m17s 10.3% words/sec/thread:   48707 lr:  0.044855 avg.loss:  1.757408 ETA:   0h 5m16s 10.8% words/sec/thread:   48309 lr:  0.044582 avg.loss:  1.754991 ETA:   0h 5m17s 12.4% words/sec/thread:   47559 lr:  0.043787 avg.loss:  1.742075 ETA:   0h 5m16s 16.2% words/sec/thread:   4

In [14]:
res = ft.predict(['بسکتبال', 'توپ'], dataset.loc[50000:52000])
res

,date,title,intro,body,category-PER,category-ENG
51505,دوشنبه ۱۱ مرداد ۱۴۰۰ - ۱۴:۳۱,تصاویر | اشک‌های گرایی پس از شکست در نیمه نهایی,سعید گرایی که با یک فن جذاب از سد حریف کروات گ...,رقابت گرایی با حریف مجارستانی\n\n\n\nرقابت گرا...,"['ورزش', 'کشتی و وزنه\u200cبرداری']",Sport
50049,سه‌شنبه ۲۵ خرداد ۱۴۰۰ - ۲۰:۱۵,هشدار بازیکنان عراق به هواداران در خصوص بازی ب...,بازیکنان تیم ملی عراق در فضای مجازی به هوادارا...,به گزارش همشهری آنلاین ایران در حالی باید تا س...,"['ورزش', 'فوتبال ايران']",Sport
51949,پنجشنبه ۲۵ شهریور ۱۴۰۰ - ۱۸:۰۵,عکس | وحید امیری کنار ستاره سرشناس جهان روی صف...,صفحه رسمی جام جهانی مقایسه‌ای میان ستاره تیم م...,به گزارش همشهری‌آنلاین، صفحه رسمی جام جهانی فی...,"['ورزش', 'فوتبال ايران']",Sport
50950,چهارشنبه ۹ تیر ۱۴۰۰ - ۱۸:۳۰,داور زن ایرانی جام جهانی را از دست داد,داور زن ایرانی یکی از نامزدهای قضاوت در جام فو...,به گزارش همشهری آنلاین فتحی که در مسابقات انتخ...,"['ورزش', 'فوتبال جهان']",Sport
50489,پنجشنبه ۳۱ تیر ۱۴۰۰ - ۲۱:۱۴,عکس | انتقاد علی دایی از شرایط ایجاد شده در خو...,علی دایی نسبت به وضعیت خوزستان انتقاد کرد.,به گزارش همشهری آنلاین علی دایی کاپیتان سابق ت...,"['ورزش', 'فوتبال ايران']",Sport
